<a href="https://colab.research.google.com/github/financieras/derivados2022/blob/main/derivados02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calcular el precio de un bono
Puede consultar el archivo de Excel: [excel_para_pyhton.xlsm](https://www.dropbox.com/s/wzq2y2k97c1d6rg/excel_para_pyhton.xlsm?dl=1)

## Precio utilizando la TIR
Supongamos un bono con las siguientes características:
* Nominal 1.000 €
* Cupón anual 10%
* Madura al quinto año
* TIR: r = 8%

Se pide:
* Calcular el precio del bono
* Una vez conocido el precio del bono, comprobar que la TIR coincide con el dato proporcionado



In [ ]:
pip install numpy-financial

In [ ]:
import numpy_financial as npf

# DATOS DEL BONO
nominal = 1000
cupon = 0.1
n = 5  # años
r = 0.08

# FLUJOS DE CAJA
flujos = [0]
for i in range(1, n+1):
    if i < n:
        flujos.append(cupon * nominal)
    elif i == n:
        flujos.append(cupon * nominal + nominal)
print("Flujos de caja: ", flujos)

# PRECIO DEL BONO
precio = 0
for i in range(1, n+1):
    precio += flujos[i] / (1+r)**i
print("Precio del bono:", precio)
bono = flujos[:]
bono[0] = -precio

# COMPROBACIÓN TIR = r
tir = npf.irr(bono)
print(f"La TIR del bono es: {tir:.5%}")  # Internal rate of return

## Precio utilizando la ETTI
Supongamos un bono con las siguientes características:
* Nominal 1.000 €
* Cupón anual 10%
* Madura al quinto año
* La ETTI para los diferentes años es:
 - año 1: 2%
 - año 2: 4%
 - año 3: 6%
 - año 4: 8%
 - año 5: 10%

Se pide:
* Calcular el precio del bono
* Calcular la TIR

In [ ]:
import numpy_financial as npf

# DATOS
nominal = 1000
cupon = 0.1
n = 5 # años
r01 = 0.02
r02 = 0.04
r03 = 0.06
r04 = 0.08
r05 = 0.10
etti = [r01,r02,r03,r04,r05]

# FLUJOS DE CAJA
flujos = [0]
for i in range(1, n+1):
    if i < n:
        flujos.append(cupon * nominal)
    elif i == n:
        flujos.append(cupon * nominal + nominal)
print("Flujos de caja: ", flujos)

# PRECIO DEL BONO CON LA ETTI
precio = 0
for i in range(1, n+1):
    precio += flujos[i] / (1+etti[i-1])**i
print("Precio del bono:", precio)
bono = flujos[:]
bono[0] = -precio

# CALCULAR LA TIR DEL BONO TIR = r
tir = npf.irr(bono)
print(f"La TIR del bono es: {tir:.6%}")  # Internal rate of return

# Creación de un bono sintético cupón cero
Supongamos dos bonos A y B, ambos a 5 años con las siguientes características:
* Bono A
 - Nominal: 10.000 €
 - Cupón anual: 2%
* Bono B
 - Nominal: 4.000 €
 - Cupón anual: 25%
* La ETTI para los diferentes años es:
 - año 1: 2%
 - año 2: 4%
 - año 3: 6%
 - año 4: 8%
 - año 5: 10%

Se pide:
* Calcular el precio de ambos bonos
* Calcular la TIR de ambos bonos
* Crear el bono C que es un bono sintético que se forma combinando los bonos A y B para conseguir un bono cupón cero a 5 años.
* Calcular la TIR del bono C y comprobar que es igual a la ETTI a 5 años, que es del 10%.

In [ ]:
import numpy_financial as npf

# DATOS BONO A
nominalA = 10_000
cuponA = 0.02
n = 5 # años, para A y B
# DATOS BONO B
nominalB = 4_000
cuponB = 0.25
# ETTI
etti =[.02,.04,.06,.08,.10]

# FLUJOS DE CAJA DE LOS BONOS
def flujos(nominal, cupon, n):
    cf = [0]  # array con el cash flow
    for i in range(1, n+1):
        if i < n:
            cf.append(cupon * nominal)
        elif i == n:
            cf.append(cupon * nominal + nominal)
    return cf

# Flujos de caja de los Bonos A y B
flujosA = flujos(nominalA, cuponA, n)
flujosB = flujos(nominalB, cuponB, n)

# PRECIO DE UN BONO CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, n+1):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

# Precios de los Bonos A y B
precioA = precio_etti(flujosA, etti)
precioB = precio_etti(flujosB, etti)
print(f"El precio del bono A es {precioA}")
print(f"El precio del bono B es {precioB}")
# Flujos de caja de los bonos A y B
bonoA = flujosA[:]
bonoA[0] = -precioA
bonoB = flujosB[:]
bonoB[0] = -precioB

# Creación del bono sintético C
# m es el número de bonos que se han de comprar o vender de uno de los tipos
#m = (cuponB*nominalB)/(cuponA*nominalA)
m = max((cuponA*nominalA)/(cuponB*nominalB), (cuponB*nominalB)/(cuponA*nominalA))
# Flujos de caja del bono C
bonoC = [0]*(n+1)
for i in range(0, n+1):
    bonoC[i] = m*bonoA[i] - bonoB[i]
print(f"Flujos de caja del bono C: {bonoC}")

# Calcular la TIR de los bonos
tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
tirC = npf.irr(bonoC)
print(f"La TIR del bono A es: {tirA:.6%}")
print(f"La TIR del bono B es: {tirB:.6%}")
print(f"La TIR del bono C es: {tirC:.6%}")

# Reinversión de flujos de caja intermedios
La importancia de trabajar con bonos cupón cero.

La TIR es la Tasa Interna de Rentabilidad, la palabra 'Interna' indica que si la operación financiera analizada se mezcla con otras operaciones la rentabilidad prometida por la TIR se podría alterar.

Una operación de inversión que tenga flujos de caja intermedios, por ejemplo un bono cupón explícito, puede no proporcionar a su propietario la rentabilidad que promete la TIR del bono si el inversor no se preocupa de reinvertir los flujos de caja intermedios hasta el final de la operación.

Supongamos un bono cupón explícito con las siguientes características:
* Nominal 10.000 €
* Cupón 10%
* Madura a los 10 años
* Precio de adquisición 10.000 €

Se pide:
* Calcular la TIR del bono
* Calcular la rentabilidad del inversor supueto que reinvierta los flujos de caja intermedios hasta el momento de vencimiento, a las siguientes rentabilidades:
 - al 0%
 - al 10%
 - al 20%
* Crear un bono cupón cero, al mismo plazo, por el mismo precio, cuyo último flujo de caja sea el necesario para proporcionar una rentabilidad del 10% y comprobar que su TIR es de 10%.

In [ ]:
import numpy_financial as npf

# DATOS
nominal = 10_000
cupon = .1
n = 10 # años
precio = 10_000

# Flujos de caja del bono
cf = [0]*(n+1)
for i in range(1,n+1):
    cf[i] = cupon * nominal
cf[n] += nominal
cf[0] = -precio

print(f"La TIR del bono es: {npf.irr(cf):.2%}")

# Montante de la Reinversión
def montante(cf,r):
    m = 0 # montante
    for i in range(1, n+1):
        m += cf[i]*(1+r)**(n-i)
    return m

tasas_reinversion = [0,.1,.2]
for t in tasas_reinversion:
    m = montante(cf,t)
    print()
    print(f"El montante reinvirtiendo al tanto del {t:.0%} es {m:,.2f} €")
    print(f"La rentabilidad del inversor reinvirtiendo al tanto {t} es {(m/precio)**(1/n)-1:.2%}")

# Comparación entre una inversión cierta y otra aleatoria
Comparemos dos inversiones A y B, ambas de duración 1 año y que proporcionan ambas un 10% anual, con una inversión de un millón de euros cada una. La inversión A es cierta, esto supone que existe un 100% de probabilidad de que el flujo de caja final sea 1.100.000 €. La inversión B, tiene una probabilidad del 1% de que el montante final sea m1, y una probabilidad del 99% de que el montante final sea m2.  
Se pide:  
* Si m1 = 0, calcular m2, para que la rentabilidad esperada de la inversión B sea del 10%
* Si m1 = 500.000, calcular m2, para que la rentabildidad esperada de B sea el 10%
* Comprobar que la TIR de ambas inversiones es del 10%

# Forward $r_{12}$ conocidos dos bonos
Dados los bonos A y B calcular la TIR del bono C.
* El bono A es un bono cupón cero de duración un año y TIR del 10%
* El bono B es un bono cupón explícito de 4.400 € anuales,  nominal 10.000 €, duración 2 años y precio de 14.000 €
* El bono C es un bono cupón cero a dos años, que se obtiene como bono sintético combinando los bonos A y B

In [ ]:
import numpy_financial as npf

nominalA = 8800
bonoB = [-14000, 4400, 14400]
bonoA = [-nominalA/1.1, nominalA, 0]
m = bonoA[1]/bonoB[1]
print(f"El multiplicador m es {m}")
bonoC = [m*bonoB[0]-bonoA[0], m*bonoB[1]-bonoA[1], m*bonoB[2]-bonoA[2]]
print("Bono C: ", bonoC)
tirC = npf.irr(bonoC)
print(f"La TIR del bono C es: {tirC:.6%}")